In [30]:
import time
import pandas as pd
from bs4 import BeautifulSoup
# pip install selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
#from selenium.webdriver.support.select import Select  (not necessary now)

In [31]:
# a helper function to check if the button exists on the page
def check_exists_by_xpath(xpath):
    try:
        browser.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

In [32]:
# a helper function to show more articles
def click_more_article():
    more_article_exists = check_exists_by_xpath('//button[@class="button__e30d9cff secondary__ac37e1a2"]')
    if more_article_exists:
        browser.find_element_by_xpath('//button[@class="button__e30d9cff secondary__ac37e1a2"]').click()
        time.sleep(5)

In [33]:
# bloomberg website when searching 'electric vehicle'
path = "selenium/chromedriver"
link = "https://www.bloomberg.com/search?query=electric%20vehicle"
#go to bloomberg and search the new key word and copy-paste the weblink here

In [34]:
# start bloomberg page
browser = webdriver.Chrome(path)
browser.get(link)
time.sleep(2)

In [35]:
#set up filters

browser.find_element_by_xpath('//a[contains(text(), "Stories")]').click()
time.sleep(2)
browser.find_element_by_xpath('//a[contains(text(), "By Newest")]').click()
time.sleep(2)

In [40]:
page_num = 900 # click 'more articles' 900 times

headlines = []
authors = []
publish_date = []
links = []


for i in range(0, page_num):
    #show more articles
    click_more_article()  
    
# parse to a soup
page_source = browser.page_source
soup = BeautifulSoup(page_source, 'lxml')
stories_content = soup.find_all('div', class_="text__1793994f withThumbnail__deb7b221")

# extract the headline, authors, publish_date, and link 
for story in stories_content:
    headline = story.find('a', class_='headline__55bd5397').text
    node = story.find('div', class_='authors__70a84826')
    if node is not None:
        author = node.text
    else:
        author = 'null'
    #author = story.find('div', class_='authors__70a84826').text
    date = story.find('div', class_='publishedAt__79f8aaad').text
    link = story.find('a', class_='headline__55bd5397')['href']

    # append to our accumulative lists
    headlines.append(headline)
    authors.append(author)
    publish_date.append(date)
    links.append(link)        


In [41]:
data = {
    'headlines':headlines,
    'authors': authors,
    'publish_date':publish_date,
    'links':links
}


df = pd.DataFrame(data)

# check the shape of our dataframe
df.shape

(8579, 4)

In [42]:
# take a glimpse 
df.head()

,headlines,authors,publish_date,links
0,Mercedes Drops More Hints About Its Sports Car...,By Christoph Rauwald,"October 23, 2020",https://www.bloomberg.com/news/articles/2020-1...
1,Electric-Car Mania Turns Battery Makers Into P...,By Dana Hull and Gabrielle Coppola,"October 23, 2020",https://www.bloomberg.com/news/articles/2020-1...
2,Canada’s Stock Winners and Losers in U.S. Vote...,By Divya Balji,"October 23, 2020",https://www.bloomberg.com/news/articles/2020-1...
3,"Tesla Is Recalling About 30,000 Imported Vehic...",By Charlie Zhu and Chunying Zhang,"October 23, 2020",https://www.bloomberg.com/news/articles/2020-1...
4,Renault Beats Revenue Estimates as Electric-Ca...,By Tara Patel,"October 23, 2020",https://www.bloomberg.com/news/articles/2020-1...


In [43]:
df.to_csv('electric_vehicle1.csv',index=True)

In [44]:
# close the browser
browser.quit()